###  贝叶斯拼写检查器 ###

In [18]:
import re, collections
 
def words(text): return re.findall('[a-z]+', text.lower()) 
 
def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model
 
NWORDS = train(words(open('big.txt').read()))
 
alphabet = 'abcdefghijklmnopqrstuvwxyz'
 
def edits1(word):
    n = len(word)
    return set([word[0:i] + word[i+1:] for i in range(n)] +                     # deletion
               [word[0:i] + word[i+1] + word[i] + word[i+2:] for i in range(n-1)] + # transposition
               [word[0:i] + c + word[i+1:] for i in range(n) for c in alphabet] + # alteration
               [word[0:i] + c + word[i:] for i in range(n+1) for c in alphabet])  # insertion
 
def known_edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS}
 
def known(words):
    return {w for w in words if w in NWORDS}
 
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])

In [19]:
#appl #appla #learw #tess #morw
correct('knon')

'know'

### 求解：argmaxc P(c|w) -> argmaxc P(w|c) P(c) / P(w) ###

* P(c), 文章中出现一个正确拼写词 c 的概率, 也就是说, 在英语文章中, c 出现的概率有多大
* P(w|c), 在用户想键入 c 的情况下敲成 w 的概率. 因为这个是代表用户会以多大的概率把 c 敲错成 w
* argmaxc, 用来枚举所有可能的 c 并且选取概率最大的

In [8]:
# 把语料中的单词全部抽取出来, 转成小写, 并且去除单词中间的特殊符号
def words(text): return re.findall('[a-z]+', text.lower()) 
 
def train(features):
    # 默认值为1的dict
    model = collections.defaultdict(lambda: 1)
    # 计数
    for f in features:
        model[f] += 1
    return model
 
NWORDS = train(words(open('big.txt').read()))


要是遇到我们从来没有过见过的新词怎么办. 假如说一个词拼写完全正确, 但是语料库中没有包含这个词, 从而这个词也永远不会出现在训练集中. 于是, 我们就要返回出现这个词的概率是0. 这个情况不太妙, 因为概率为0这个代表了这个事件绝对不可能发生, 而在我们的概率模型中, 我们期望用一个很小的概率来代表这种情况. lambda: 1

In [17]:
NWORDS
sorted(NWORDS.items(), key=lambda d: -d[1])

[('the', 80031),
 ('of', 40026),
 ('and', 38313),
 ('to', 28767),
 ('in', 22048),
 ('a', 21156),
 ('that', 12513),
 ('he', 12402),
 ('was', 11411),
 ('it', 10682),
 ('his', 10035),
 ('is', 9775),
 ('with', 9741),
 ('as', 8065),
 ('i', 7683),
 ('had', 7384),
 ('for', 6940),
 ('at', 6792),
 ('by', 6739),
 ('on', 6644),
 ('not', 6626),
 ('be', 6156),
 ('from', 5710),
 ('but', 5654),
 ('s', 5632),
 ('you', 5623),
 ('or', 5353),
 ('her', 5285),
 ('him', 5231),
 ('which', 4843),
 ('were', 4290),
 ('all', 4145),
 ('this', 4064),
 ('she', 3947),
 ('they', 3939),
 ('are', 3631),
 ('have', 3494),
 ('said', 3465),
 ('an', 3424),
 ('one', 3372),
 ('who', 3051),
 ('so', 3018),
 ('what', 3012),
 ('there', 2973),
 ('their', 2956),
 ('when', 2924),
 ('been', 2600),
 ('may', 2552),
 ('if', 2373),
 ('no', 2349),
 ('up', 2285),
 ('my', 2250),
 ('them', 2242),
 ('into', 2125),
 ('more', 1998),
 ('out', 1988),
 ('pierre', 1965),
 ('would', 1954),
 ('prince', 1936),
 ('me', 1921),
 ('we', 1907),
 ('did', 18

###  编辑距离: ###
两个词之间的编辑距离定义为,使用了几次
1. 插入(在词中插入一个单字母), 

2. 删除(删除一个单字母), 

3. 交换(交换相邻两个字母), 

4. 替换(把一个字母换成另一个)

的操作从一个词变到另一个词.

In [8]:
#生成所有与单词 w 编辑距离为 1 的单词的集合
def edits1(word):
    n = len(word)
    return set([word[0:i]+word[i+1:] for i in range(n)] +                     # deletion , 去掉第i个字符
               [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] + # transposition , 交换i和i+1个字符
               [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] + # alteration , 替换第i个字符
               [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])  # insertion , 插入到第i个字符的位置

与 something 编辑距离为2的单词居然达到了 114,324 个

优化:在这些编辑距离小于2的词中间, 只把那些正确的词作为候选词,只能返回 3 个单词: ‘smoothing’, ‘something’ 和 ‘soothing’

In [9]:
#生成所有与单词 w 编辑距离为 2 的单词的集合
#在这些编辑距离小于2的词中间, 只把那些正确的词作为候选词
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

语料库中词语 > 在语料库中的，编辑距离为1的词语 > 在语料库中的，编辑距离为2的词语 > 没出现过，原样输出

最终输出结果为：经过上述筛选后的所有候选词中，在语料库中概率最大的词。

In [10]:
def known(words): return set(w for w in words if w in NWORDS)

#如果known(set)非空, candidate 就会选取这个集合, 而不继续计算后面的
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])